# Distributional Semantics Model: Narrow Word-Window (-2,+2)

### A. Data-Loading Facilities

In [4]:
import os
os.chdir("/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/BASIC_TOPICS/DISTRIBUTIONAL_SEMANTICS/ASSIGNMENT_03")

In [5]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from collections import defaultdict
from functools import partial
from itertools import permutations, product
punctuation = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [37]:
def load_wiki(cutoffFreq=50):
    
    print "... extracting data"
    with open('wikicorpus.txt','rb') as f:
        raw = f.readlines()
    raw = [sent[4:].split() for sent in raw if sent.startswith('<c>')] 
        # sent[4:]: get rid of initial <c>.
        # extract sentences; split sentences into word complexes.
    raw = [[map(partial(str.split, word), '|') for word in sent] for sent in raw] 
        # split word complexes into words.
    
    print "... cleaning data"
    sents = [[word[0][1].lower() for word in sent if len(word[0])>1 
              and (word[0][2].startswith('N') or word[0][2]=='.')
              and word[0][1].lower() not in stopwords
              and word[0][1] not in punctuation] for sent in raw]
        # extract lemmas => complete sents corpus .
    
    print "... building token list and vocabulary"
    tokens = [word for sent in sents for word in sent]
        # type: list of words.
    fdist = nltk.FreqDist(tokens)
    vocab = list(set(tokens))   
    
    print "... saving top %d-frequent in vocabulary" % cutoffFreq
    vocab = [word for word in vocab if fdist[word] >= cutoffFreq]
        # vocab is not returned, because the k-frequent cut latter can change it.
    sents = [word.decode('utf-8','ignore') for sent in sents for word in sent if word in vocab]
        # type: list of all words in the corpus, separated by periods.
    sents.append('.') # handle -1 window lookahead +2 error.
        
    return sents

In [38]:
%%time
sents = load_wiki(50)

... extracting data
... cleaning data
... building token list and vocabulary
... saving top 50-frequent in vocabulary
CPU times: user 5min 9s, sys: 6.89 s, total: 5min 16s
Wall time: 5min 14s


### B1. Analyzer: Cooccurrence Matrix Based

#### A. Model

In [39]:
import numpy as np

In [40]:
# SIMILARITY MEASURES
def cosine(w2w):
    w2w_norm = w2w / np.apply_along_axis(lambda r: np.sqrt(np.dot(r,r))
                               , 1, w2w)[:,np.newaxis]
    return np.dot(w2w_norm, w2w_norm.T)
    
def ppmi(w2w):
    rowSums, colSums, totalSums = w2w.sum(axis=1), w2w.sum(axis=0), w2w.sum()
    pwi, pwj, ppmiMatrix = rowSums/totalSums, colSums/totalSums, w2w/totalSums
    ppmiMatrix /= pwi[:,np.newaxis] # * 1/pwi by row.
    ppmiMatrix /= pwj # * 1/pwj by col.
    ppmiMatrix = np.nan_to_num(np.log(ppmiMatrix)) # compute pmi.
    ppmiMatrix = np.maximum(ppmiMatrix, 0) # compute ppmi.
    return ppmiMatrix

In [41]:
class SimpleDistSem:
    
    def __init__(self, data=sents, kFrequent=50):
        self.sents = sents # a list of all words in the corpus (count=2573841).
        self.vocab = list({word for word in sents})
        self.wordToIndex = {word:i for i,word in enumerate(self.vocab)}
        self.indexToWord = {i:word for word,i in self.wordToIndex.iteritems()}
    
    def build_w2w_matrix(self):
        
        print "... counting words"
        cooccurrenceDict = defaultdict(int)
        for i,word in enumerate(self.sents):
            context = []
            if word=='.': continue # not pass, because we do not execute anything after this.
            elif self.sents[i+2]=='.':
                context.extend([word,self.sents[i-2],self.sents[i-1],self.sents[i+1]])
            elif self.sents[i+1]=='.':
                context.extend([word,self.sents[i-2],self.sents[i-1]])
            elif self.sents[i-1]=='.':
                context.extend([word,self.sents[i+1],self.sents[i+2]])
            elif self.sents[i-2]=='.':
                context.extend([word,self.sents[i-1],self.sents[i+1],self.sents[i+2]])
            elif i==0: context.extend([word,self.sents[i+1],self.sents[i+2]])
            elif i==1: context.extend([word,self.sents[i-1],self.sents[i+1],self.sents[i+2]])
            else: 
                context.extend([word,self.sents[i-2],self.sents[i-1],self.sents[i+1],self.sents[i+2]])
            for w_i,w_j in product(context,repeat=2):
                cooccurrenceDict[(self.wordToIndex[w_i],self.wordToIndex[w_j])] += 1
        
        print "... building cooccurrence matrix"
        self.w2w = np.zeros((len(self.vocab),len(self.vocab)))
        for (widx_i,widx_j),count in cooccurrenceDict.iteritems():
            self.w2w[widx_i][widx_j] = count   
    
    def build_similarity_matrix(self, similarity=ppmi):
        self.simMatrix = similarity(self.w2w)
    
    def k_most_similar(self, words, k=20):
        assert len(words)==len(filter(lambda w:1 if w in self.vocab else 0, words))
        w2sim = {}
        for word in words:
            simList = self.simMatrix[self.wordToIndex[word]]
            w2sim[word] = map(lambda idx:(self.indexToWord[idx],
                                          self.simMatrix[self.wordToIndex[word]][idx]),
                              np.argsort(simList)[::-1][1:k+1])
            # [1:k+1]: skip self-similarty.
        return w2sim

In [42]:
%%time
ds = SimpleDistSem(sents)

CPU times: user 397 ms, sys: 38.5 ms, total: 436 ms
Wall time: 429 ms


In [43]:
%%time
ds.build_w2w_matrix()

... counting words
... building cooccurrence matrix
CPU times: user 29.7 s, sys: 404 ms, total: 30.1 s
Wall time: 30.1 s


#### b. Evaluator: K-Frequent

In [44]:
%%time
ds.build_similarity_matrix(ppmi)

CPU times: user 1.72 s, sys: 635 ms, total: 2.36 s
Wall time: 2.37 s


In [45]:
%%time
words = ['car','bus','hospital','hotel','gun','bomb','horse','fox','table','bowl','guitar','piano']
w2sim = ds.k_most_similar(words)

CPU times: user 8.64 ms, sys: 681 µs, total: 9.32 ms
Wall time: 8.52 ms


In [46]:
w2sim['car']

[(u'bugatti', 4.7143853568662202),
 (u'brabham', 4.4911806323919503),
 (u'racing', 4.3207762030217518),
 (u'bogie', 4.2579823032574255),
 (u'bentley', 4.0246329484148369),
 (u'chassis', 3.975228905852747),
 (u'earnhardt', 3.9719011157600725),
 (u'brake', 3.9677863816476124),
 (u'audi', 3.9591815344540939),
 (u'cable', 3.9407890381346959),
 (u'motors', 3.9113066139719526),
 (u'clutch', 3.8885512886704099),
 (u'aston', 3.8125911918074733),
 (u'truck', 3.7683664258059602),
 (u'grip', 3.7399726307408159),
 (u'prix', 3.5943387129383644),
 (u'ag', 3.571643015859757),
 (u'tire', 3.528999152539019),
 (u'installation', 3.5253515144216951),
 (u'driver', 3.4699657456182273)]

In [47]:
%%time
ds.build_similarity_matrix(cosine)

CPU times: user 25.3 s, sys: 293 ms, total: 25.6 s
Wall time: 4.06 s


In [48]:
%%time
words = ['car','bus','hospital','hotel','gun','bomb','horse','fox','table','bowl','guitar','piano']
w2sim = ds.k_most_similar(words)

CPU times: user 12.5 ms, sys: 739 µs, total: 13.3 ms
Wall time: 12.3 ms


In [49]:
w2sim['piano']

[(u'sonata', 0.35508485693440395),
 (u'trio', 0.23464065535666892),
 (u'concerto', 0.22710273751375706),
 (u'violin', 0.19383107647502684),
 (u'cello', 0.1456890210312965),
 (u'pianist', 0.12456051636374897),
 (u'beethoven', 0.12177085405267193),
 (u'bartk', 0.11200192105777483),
 (u'guitar', 0.10055489337650286),
 (u'flute', 0.094643060644424987),
 (u'music', 0.09338756848588263),
 (u'piece', 0.090224149358330175),
 (u'bass', 0.085251841930458025),
 (u'quartet', 0.082935922803059489),
 (u'no.', 0.08248275742797527),
 (u'mozart', 0.082102226327779776),
 (u'shostakovich', 0.078145130150868741),
 (u'solo', 0.077701860395713432),
 (u'instrumentation', 0.076788138499475411),
 (u'debussy', 0.076491813812711223)]

#### c. Evaluator: BLESS

In [50]:
import random

In [51]:
def bless_evaluator(simMatrix=None, indexers=[None,None]):
    wordToIndex, indexToWord = indexers
    path = '/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/BASIC_TOPICS/DISTRIBUTIONAL_SEMANTICS/ASSIGNMENT_03/BLESS_part.txt'
    with open(path,'rb') as f:
        bless = f.readlines()
    bless = [line.split('\t') for line in bless] # split into (concept, _, relation, relatum).
    crPairs = [(c.split('-')[0],r.split('-')[0],rel) for c,_,rel,r in bless]
    posPairs = [(c,r) for c,r,rel in crPairs if rel=='hyper']
    negPairs = [(c,r) for c,r,rel in crPairs if rel=='mero']
    
    return [map(lambda (c,r):(c,r,simMatrix[wordToIndex[c]][wordToIndex[r]]), posPairs),
            map(lambda (c,r):(c,r,simMatrix[wordToIndex[c]][wordToIndex[r]]), negPairs)]

##### PPMI

In [52]:
%%time
ds.build_similarity_matrix(ppmi)
posEval, negEval = bless_evaluator(ds.simMatrix, indexers=[ds.wordToIndex, ds.indexToWord])

CPU times: user 1.8 s, sys: 643 ms, total: 2.44 s
Wall time: 2.45 s


In [53]:
print "Examples of Evaluation on Positive Relations (PPMI): "
print random.sample(posEval, 5)
print "Average PPMI: ", np.mean([ppmiVal for _,_,ppmiVal in posEval])

Examples of Evaluation on Positive Relations (PPMI): 
[('bomb', 'object', 0.0), ('coat', 'clothing', 0.0), ('dolphin', 'vertebrate', 0.0), ('goat', 'creature', 3.5142631417547738), ('bull', 'creature', 0.0)]
Average PPMI:  1.41877631314


In [54]:
print "Examples of Evaluation on Negative Relations (PPMI): "
print random.sample(negEval, 5)
print "Average PPMI: ", np.mean([ppmiVal for _,_,ppmiVal in negEval])

Examples of Evaluation on Negative Relations (PPMI): 
[('bull', 'tail', 0.0), ('cannon', 'wheel', 1.1155047242769605), ('pub', 'light', 0.0), ('potato', 'carbohydrate', 4.0880854372575479), ('piano', 'key', 2.3543081397559291)]
Average PPMI:  1.21180615243


##### Cosine

In [55]:
%%time
ds.build_similarity_matrix(cosine)
posEval, negEval = bless_evaluator(ds.simMatrix, indexers=[ds.wordToIndex, ds.indexToWord])

CPU times: user 25.1 s, sys: 308 ms, total: 25.4 s
Wall time: 4.22 s


In [56]:
print "Examples of Evaluation on Positive Relations (Cosine): "
print random.sample(posEval, 5)
print "Average Cosine: ", np.mean([cosineVal for _,_,cosineVal in posEval])

Examples of Evaluation on Positive Relations (Cosine): 
[('ferry', 'boat', 0.034407417781520594), ('hammer', 'tool', 0.033215358203046413), ('clarinet', 'instrument', 0.15833977866172178), ('spear', 'device', 0.004680927915039431), ('radio', 'commodity', 0.0055174583898223474)]
Average Cosine:  0.0287957750164


In [57]:
print "Examples of Evaluation on Positive Relations (Cosine): "
print random.sample(negEval, 5)
print "Average Cosine: ", np.mean([cosineVal for _,_,cosineVal in negEval])

Examples of Evaluation on Positive Relations (Cosine): 
[('bus', 'radio', 0.015169699024820705), ('car', 'frame', 0.010182680535167509), ('grape', 'juice', 0.29588658090230935), ('car', 'radio', 0.010599621325313089), ('motorcycle', 'engine', 0.14918700503215837)]
Average Cosine:  0.0219069555561
